In [1]:
import os
import json
from pathlib import Path
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import sys
import os
sys.path.append(os.path.abspath(".."))
from models.model_v2 import CNNModelV2

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROJECT_ROOT = Path(".").resolve()
DATA_DIR = "//Users/prajjvalgovil/Downloads/resmet/collaborative_cnn_team04/data/train"

MODEL_SAVE_PATH = PROJECT_ROOT / "models" / "model_v2.pth"
METRICS_SAVE_PATH = PROJECT_ROOT / "results" / "metrics_v2.json"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [3]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
])


In [4]:
class DogsCatsReduxDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform

        self.image_paths = sorted([
            p for p in self.root_dir.iterdir()
            if p.suffix.lower() in [".jpg", ".jpeg", ".png"]
        ])

        self.labels = []
        for p in self.image_paths:
            name = p.name.lower()
            if name.startswith("cat"):
                self.labels.append(0)
            elif name.startswith("dog"):
                self.labels.append(1)
            else:
                raise ValueError(f"Unknown filename: {name}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        return img, label


In [5]:
dataset = DogsCatsReduxDataset(DATA_DIR, transform=transform)

train_size = int(0.8 * len(dataset))   # 80% train
val_size = len(dataset) - train_size   # 20% validation

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))


Train samples: 20000
Val samples: 5000


In [6]:
model = CNNModelV2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

In [8]:
num_epochs = 20

train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    # TRAINING WITH PROGRESS BAR
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # VALIDATION
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = correct / total
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}]  Loss: {avg_train_loss:.4f}  Val Acc: {val_acc:.4f}")


Epoch 1/20: 100%|██████████| 313/313 [19:05<00:00,  3.66s/it]


Epoch [1/20]  Loss: 0.6638  Val Acc: 0.6880


Epoch 2/20: 100%|██████████| 313/313 [12:56<00:00,  2.48s/it]


Epoch [2/20]  Loss: 0.5814  Val Acc: 0.7374


Epoch 3/20: 100%|██████████| 313/313 [12:44<00:00,  2.44s/it]


...

Epoch [18/20]  Loss: 0.3242  Val Acc: 0.8000


Epoch 19/20: 100%|██████████| 313/313 [12:56<00:00,  2.48s/it]


Epoch [19/20]  Loss: 0.3221  Val Acc: 0.8536


Epoch 20/20: 100%|██████████| 313/313 [12:56<00:00,  2.48s/it]


Epoch [20/20]  Loss: 0.3068  Val Acc: 0.8012


In [9]:
MODEL_SAVE_PATH = "/Users/prajjvalgovil/Downloads/resmet/collaborative_cnn_team04/models/model_v2.pth"
torch.save(model.state_dict(), MODEL_SAVE_PATH)
MODEL_SAVE_PATH

'/Users/prajjvalgovil/Downloads/resmet/collaborative_cnn_team04/models/model_v2.pth'

In [10]:
metrics = {
    "model": "model_v2",
    "user": "Prajjval Govil",
    "train_loss": train_losses,
    "val_accuracy": val_accuracies,
    "epochs": num_epochs,
}
METRICS_SAVE_PATH = "/Users/prajjvalgovil/Downloads/resmet/collaborative_cnn_team04/results/metrics_v2.json"
with open(METRICS_SAVE_PATH, "w") as f:
    json.dump(metrics, f, indent=2)

METRICS_SAVE_PATH


'/Users/prajjvalgovil/Downloads/resmet/collaborative_cnn_team04/results/metrics_v2.json'